# Demo

The purpose of this notebook is to demonstrate the functionality of `flightmodels`.

The demo is organised to show

- Step 1: Regressing the Seaglider steady flight model for chosen parameters

- Step 2: Recalculating the glider flight using the flight model and chosen parameters



In [1]:
from pathlib import Path

import xarray as xr
from flightmodels import plotters, readers, tools, utilities, writers
import numpy as np
import xarray as xr
from flightmodels import seaglider, plotters  # Assuming correct project style


In [2]:

# 1. Load your dataset
glider = xr.open_dataset('../data/sg014_20080214T220104_delayed.nc')
glider = tools.calc_w_meas(glider)

# 2. Rename and clean up fields as needed
glider = glider.rename({
    'VBD_CC': 'VBD',
    'divenum': 'DIVENUM',
})


In [3]:

# 3. Add missing variables
# a) Create UPDN from PITCH
glider = glider.assign_coords(TIME=glider['TIME'])  # Make sure TIME is coordinate
glider['UPDN'] = xr.where(glider['PITCH'] > 0, 1, -1)

# b) Create VERTICAL_SPEED from dP/dt
# Assume TIME is in seconds or known units


# c) Create fake C_VBD
glider['C_VBD'] = xr.zeros_like(glider['VBD'])


In [4]:

# d) Add starter attributes
glider.attrs.update({
    'hd_a': 0.0036,     # starting guess
    'hd_b': 0.0098,     # starting guess
    'hd_c': 0.0010,     # starting guess
    'vbdbias': 0.0,     # start assuming no bias
    'abs_compress': 4e-6, # reasonable guess
    'therm_expan': 2e-5,  # reasonable guess
    'rho0': 1025.0,     # typical seawater density
    'vbd_min_cnts': 0.0,
    'vbd_cnts_per_cc': 1.0,
    'temp_ref': 10.0,
    'volmax': 0.0,
    'mass': 52.0,  # Approx mass of glider [kg]
})

# 4. Subselect a few dives to test
dives_to_use = glider['DIVENUM'].values
ensmat = dives_to_use[:20]  # first 20 dives for testing


In [5]:
glider

<xarray.Dataset> Size: 17MB
Dimensions:                               (N_MEASUREMENTS: 54404)
Coordinates:
    LONGITUDE                             (N_MEASUREMENTS) float64 435kB ...
    LATITUDE                              (N_MEASUREMENTS) float64 435kB ...
    TIME                                  (N_MEASUREMENTS) datetime64[ns] 435kB ...
    DEPTH                                 (N_MEASUREMENTS) float64 435kB ...
Dimensions without coordinates: N_MEASUREMENTS
Data variables: (12/59)
    TEMP_RAW_QC                           (N_MEASUREMENTS) float32 218kB ...
    TEMP_QC                               (N_MEASUREMENTS) float32 218kB ...
    GLIDE_SPEED_QC                        (N_MEASUREMENTS) float32 218kB ...
    TIME_DOXY                             (N_MEASUREMENTS) datetime64[ns] 435kB ...
    DOXY_QC                               (N_MEASUREMENTS) float32 218kB ...
    PSAL_RAW_QC                           (N_MEASUREMENTS) float32 218kB ...
    ...                                    ...
    SENSOR_CTD_0019,                      (N_MEASUREMENTS) float64 435kB ...
    SENSOR_DISSOLVED_GAS_SENSORS_UNKNOWN  (N_MEASUREMENTS) float64 435kB ...
    PLATFORM_SERIAL_NUMBER                <U5 20B ...
    GLIDER_VERT_VELO_DZDT                 (N_MEASUREMENTS) float64 435kB nan ...
    UPDN                                  (N_MEASUREMENTS) int64 435kB -1 ... -1
    C_VBD                                 (N_MEASUREMENTS) float32 218kB 0.0 ...
Attributes: (12/52)
    title:                                      OceanGliders trajectory file
    id:                                         sg014_20080214T220104_delayed
    platform:                                   sub-surface gliders
    platform_vocabulary:                        https://vocab.nerc.ac.uk/coll...
    naming_authority:                           edu.washington.apl
    institution:                                School of Oceanography\nUnive...
    ...                                         ...
    rho0:                                       1025.0
    vbd_min_cnts:                               0.0
    vbd_cnts_per_cc:                            1.0
    temp_ref:                                   10.0
    volmax:                                     0.0
    mass:                                       52.0

In [6]:

# 5. Which parameters to optimize
whichpar = [1, 1, 1, 0, 0, 0]  # solve for hd_a, hd_b, vbdbias

# 6. Run the regression
regressout, allwrms = seaglider.regress_all_vec(
    whichpar=whichpar,
    glider=glider,
    whichone=10,  # Ramsey's method
    ensmat=ensmat,
    plotflag=False,
    unstdyflag=0,
)

def renormalize_params(params, factors, direction='normalize'):
    """
    Renormalize parameters with given factors.

    Parameters
    ----------
    params : np.ndarray
        The parameters to transform. Expected order is
        [hd_a, hd_b, vbdbias, abs_compress, therm_expan, hd_c].
    factors : list or np.ndarray
        The factors for renormalization.
    direction : str, optional
        'normalize' to divide by factors, 'denormalize' to multiply by factors.
        Default is 'normalize'.

    Returns
    -------
    np.ndarray
        Transformed parameters.

    Raises
    ------
    ValueError
        If the direction is not 'normalize' or 'denormalize'.
    """
    if direction == 'normalize':
        return params * factors
    elif direction == 'denormalize':
        return params / factors
    else:
        raise ValueError("Direction must be 'normalize' or 'denormalize'.")

# Define renormalization factors
factors = np.array([1e3, 1e3, 1, 1e6, 1e5, 1e5])

# Create a vector of params from regressout with the same size as factors
params = np.zeros_like(factors)
params[np.array(whichpar, dtype=bool)] = regressout
# Renormalize regressout
#regressout_normalized = renormalize_params(regressout, factors, direction='normalize')
regressout_norm = renormalize_params(params, factors, direction='denormalize')

print("Optimized parameters:", regressout_norm)
print("Final WRMS:", allwrms)



Doing regress_all_vec
1.312e+06 0.0036 0.0098 0 4e-06 2e-05 0.001
1.313e+06 0.00378 0.0098 0 4e-06 2e-05 0.001
1.229e+06 0.0036 0.01029 0 4e-06 2e-05 0.001
1.312e+06 0.0036 0.0098 0.00025 4e-06 2e-05 0.001
1.255e+06 0.00342 0.01013 0.0001667 4e-06 2e-05 0.001
1.22e+06 0.00348 0.01034 -0.0001389 4e-06 2e-05 0.001
1.177e+06 0.00342 0.01062 -0.0003333 4e-06 2e-05 0.001
1.135e+06 0.00336 0.01089 -0.0001111 4e-06 2e-05 0.001
1.062e+06 0.00324 0.01143 -0.0001667 4e-06 2e-05 0.001
1.063e+06 0.00342 0.01143 -0.0005 4e-06 2e-05 0.001
9.912e+05 0.00312 0.01203 -0.0006667 4e-06 2e-05 0.001
9.009e+05 0.00288 0.0129 -0.001 4e-06 2e-05 0.001
8.715e+05 0.00294 0.01323 -0.0007778 4e-06 2e-05 0.001
7.661e+05 0.0027 0.01454 -0.001 4e-06 2e-05 0.001
7.678e+05 0.00246 0.01448 -0.0009444 4e-06 2e-05 0.001
6.378e+05 0.00212 0.01651 -0.001796 4e-06 2e-05 0.001
0.02828 0.00156 0.01906 -0.002611 4e-06 2e-05 0.001
0.02828 0.0016 0.01915 -0.002037 4e-06 2e-05 0.001
0.02828 0.001447 0.02068 -0.002821 4e-06 2e-05 

In [7]:
if 0:
    # And your glider dataset is called `glider`
    plotters.plot_model_comparisons(
        glider=glider,
        models={
            'steady': spd0,
            'unsteady_lag': spd['unstdy_lag'],
            'unsteady_ode': spd['unstdy_ode']
        },
        x_axis='TIME',
        select_timerange=(glider['TIME'].values[0], glider['TIME'].values[100]),
        variable='VERTICAL_SPEED',
        label_variable='Observed Vertical Speed',
        model_labels=['Flightvec0 (steady)', 'Flightvec_unstdy (lag)', 'Flightvec_unstdy (ODE)'],
        title='Comparison of Glider Flight Models'
    )